# Calibration procedure (six-steps procedures)

In [1]:
import pandas as pd
import numpy as np
import agentpy as ap

## definition des secteurs institutionnelles, des stocks et des flux

In [2]:
# definition des secteurs institutionnels
account_keys = ['H', 'F', 'B', 'G', 'CB']
account_names = ['Households', 'Firms', 'Banks', 'Government', 'Central Bank']

# definitions des regions et des secteurs productifs
regions = ['a', 'b']
sectors = [1, 2, 3]

# definitions des stocks
stock_keys = ['M', 'A', 'D', 'B', 'L']
stock_names = ['HP Money', 'Cash Advances', 'Deposits', 'Bonds', 'Loans']

# definitions des flux
flow_keys = ['C', 'W', 'Z', 'T', 'iota_A', 'iota_B', 'iota_L', 'iota_D', 'pi_d', 'pi', 
             'DeltaA', 'DeltaB', 'DeltaM', 'DeltaL', 'DeltaD']
flow_names = ['Consumption', 'Wages', 'Doles', 'Taxes', 
              'Int. on advances', 'Int. on bonds', 'Int. on loans', 'Int on deposits',
              'Entrepreneurial Profits', 'Central Bank Profits', 
              'Var. of advances', 'Var. of bonds', 'Var. of HP Money', 'var. of loans', 'Var. of deposits']


## fonctions d'initialisation des matrices

In [3]:
def create_matrices():
    stock_matrix = pd.DataFrame(0.0, index=stock_keys, columns=account_keys)
    flow_matrix = pd.DataFrame(0.0, index=flow_keys, columns=account_keys)
    return stock_matrix, flow_matrix

def print_matrices(matrices):
    f = lambda v: round(v, 2)
    print('stocks\n', matrices[0].map(f))
    print('\nflows\n', matrices[1].map(f))

In [4]:
print_matrices(create_matrices())

stocks
      H    F    B    G   CB
M  0.0  0.0  0.0  0.0  0.0
A  0.0  0.0  0.0  0.0  0.0
D  0.0  0.0  0.0  0.0  0.0
B  0.0  0.0  0.0  0.0  0.0
L  0.0  0.0  0.0  0.0  0.0

flows
           H    F    B    G   CB
C       0.0  0.0  0.0  0.0  0.0
W       0.0  0.0  0.0  0.0  0.0
Z       0.0  0.0  0.0  0.0  0.0
T       0.0  0.0  0.0  0.0  0.0
iota_A  0.0  0.0  0.0  0.0  0.0
iota_B  0.0  0.0  0.0  0.0  0.0
iota_L  0.0  0.0  0.0  0.0  0.0
iota_D  0.0  0.0  0.0  0.0  0.0
pi_d    0.0  0.0  0.0  0.0  0.0
pi      0.0  0.0  0.0  0.0  0.0
DeltaA  0.0  0.0  0.0  0.0  0.0
DeltaB  0.0  0.0  0.0  0.0  0.0
DeltaM  0.0  0.0  0.0  0.0  0.0
DeltaL  0.0  0.0  0.0  0.0  0.0
DeltaD  0.0  0.0  0.0  0.0  0.0


In [5]:
def fill_initial_params(params):
    h_vars = ['M_H', 'D_H', 'C', 'W_H', 'Z_H', 'T_H', 'iota_DH', 'pi_dH', 'DeltaM_H', 'DeltaD_H']
    f_vars = ['M_F', 'D_F', 'L_F', 'Q', 'W_F', 'T_F', 'iota_LF', 'iota_DF', 'pi_dF', 'DeltaM_F', 'DeltaL_F', 'DeltaD_F']
    b_vars = ['M_B', 'A_B', 'D_B', 'B_B', 'L_B', 'T_B', 
              'iota_AB', 'iota_BB', 'iota_LB', 'iota_DB', 'pi_dB',
              'DeltaA_B', 'DeltaB_B', 'DeltaM_B', 'DeltaL_B', 'DeltaD_B']
    g_vars = ['M_G', 'B_G', 'W_G', 'Z_G', 'T_G', 'iota_BG', 'pi_G', 'DeltaB_G', 'DeltaM_G']
    cb_vars = ['M_CB', 'A_CB', 'B_CB', 'iota_ACB', 'iota_BCB', 'pi_CB', 'DeltaA_CB', 'DeltaB_CB', 'DeltaM_CB']
    
    p = params.copy()
    for z in regions:
        for v in h_vars:
            p[f'{v}{z}'] = 0
    for s in sectors:
        for v in f_vars:
            p[f'{v}{s}'] = 0
    for v in b_vars + g_vars + cb_vars:
        p[v] = 0
    return p
            

In [6]:
params = {}
fill_initial_params(params)

{'M_Ha': 0,
 'D_Ha': 0,
 'Ca': 0,
 'W_Ha': 0,
 'Z_Ha': 0,
 'T_Ha': 0,
 'iota_DHa': 0,
 'pi_dHa': 0,
 'DeltaM_Ha': 0,
 'DeltaD_Ha': 0,
 'M_Hb': 0,
 'D_Hb': 0,
 'Cb': 0,
 'W_Hb': 0,
 'Z_Hb': 0,
 'T_Hb': 0,
 'iota_DHb': 0,
 'pi_dHb': 0,
 'DeltaM_Hb': 0,
 'DeltaD_Hb': 0,
 'M_F1': 0,
 'D_F1': 0,
 'L_F1': 0,
 'Q1': 0,
 'W_F1': 0,
 'T_F1': 0,
 'iota_LF1': 0,
 'iota_DF1': 0,
 'pi_dF1': 0,
 'DeltaM_F1': 0,
 'DeltaL_F1': 0,
 'DeltaD_F1': 0,
 'M_F2': 0,
 'D_F2': 0,
 'L_F2': 0,
 'Q2': 0,
 'W_F2': 0,
 'T_F2': 0,
 'iota_LF2': 0,
 'iota_DF2': 0,
 'pi_dF2': 0,
 'DeltaM_F2': 0,
 'DeltaL_F2': 0,
 'DeltaD_F2': 0,
 'M_F3': 0,
 'D_F3': 0,
 'L_F3': 0,
 'Q3': 0,
 'W_F3': 0,
 'T_F3': 0,
 'iota_LF3': 0,
 'iota_DF3': 0,
 'pi_dF3': 0,
 'DeltaM_F3': 0,
 'DeltaL_F3': 0,
 'DeltaD_F3': 0,
 'M_B': 0,
 'A_B': 0,
 'D_B': 0,
 'B_B': 0,
 'L_B': 0,
 'T_B': 0,
 'iota_AB': 0,
 'iota_BB': 0,
 'iota_LB': 0,
 'iota_DB': 0,
 'pi_dB': 0,
 'DeltaA_B': 0,
 'DeltaB_B': 0,
 'DeltaM_B': 0,
 'DeltaL_B': 0,
 'DeltaD_B': 0,
 'M_G': 0,
 

## fonctions d'aggregation des matrices

In [7]:
def agg_var(params, prefix):
    return sum([v for k, v in params.items() if k.startswith(prefix)])

In [8]:
params = {'x_11':1, 'x_12':2, 'y_10':0}
agg_var(params, 'x')

3

In [9]:
def calc_initial_matrices(params):
    stock_matrix, flow_matrix = create_matrices()
    stock_matrix.loc['M', 'H'] = agg_var(params, 'M_H')
    stock_matrix.loc['D', 'H'] = agg_var(params, 'D_H')
    stock_matrix.loc['M', 'F'] = agg_var(params, 'M_F')
    stock_matrix.loc['D', 'F'] = agg_var(params, 'D_F')
    stock_matrix.loc['L', 'F'] = - agg_var(params, 'L_F')
    stock_matrix.loc['M', 'B'] = params['M_B']
    stock_matrix.loc['A', 'B'] = - params['A_B']
    stock_matrix.loc['D', 'B'] = - params['D_B']
    stock_matrix.loc['B', 'B'] = params['B_B']
    stock_matrix.loc['L', 'B'] = params['L_B']
    stock_matrix.loc['M', 'G'] = params['M_G']
    stock_matrix.loc['B', 'G'] = - params['B_G']
    stock_matrix.loc['M', 'CB'] = - params['M_CB']
    stock_matrix.loc['A', 'CB'] = params['A_CB']
    stock_matrix.loc['B', 'CB'] = params['B_CB']
    stock_matrix.loc['V', :] = - stock_matrix.sum()
    stock_matrix.loc['sigma', :] = stock_matrix.sum()
    stock_matrix['sigma'] = stock_matrix.sum(axis=1)
    
    flow_matrix.loc['C', 'H'] = - agg_var(params, 'C')
    flow_matrix.loc['W', 'H'] = agg_var(params, 'W_H')
    flow_matrix.loc['Z', 'H'] = agg_var(params, 'Z_H')
    flow_matrix.loc['T', 'H'] = - agg_var(params, 'T_H')
    flow_matrix.loc['iota_D', 'H'] = agg_var(params, 'iota_DH')
    flow_matrix.loc['pi_d', 'H'] = agg_var(params, 'pi_dH')
    flow_matrix.loc['DeltaM', 'H'] = - agg_var(params, 'DeltaM_H')
    flow_matrix.loc['DeltaD', 'H'] = - agg_var(params, 'DeltaD_H')
    flow_matrix.loc['C', 'F'] = agg_var(params, 'Q')
    flow_matrix.loc['W', 'F'] = - agg_var(params, 'W_F')
    flow_matrix.loc['T', 'F'] = - agg_var(params, 'T_F')
    flow_matrix.loc['iota_L', 'F'] = - agg_var(params, 'iota_LF')
    flow_matrix.loc['iota_D', 'F'] = agg_var(params, 'iota_DF')
    flow_matrix.loc['pi_d', 'F'] = - agg_var(params, 'pi_dF')
    flow_matrix.loc['DeltaM', 'F'] = - agg_var(params, 'DeltaM_F')
    flow_matrix.loc['DeltaL', 'F'] = agg_var(params, 'DeltaL_F')
    flow_matrix.loc['DeltaD', 'F'] = - agg_var(params, 'DeltaD_F')
    flow_matrix.loc['T', 'B'] = - params['T_B']
    flow_matrix.loc['iota_A', 'B'] = - params['iota_AB']
    flow_matrix.loc['iota_B', 'B'] = params['iota_BB']
    flow_matrix.loc['iota_L', 'B'] = params['iota_LB']
    flow_matrix.loc['iota_D', 'B'] = - params['iota_DB']
    flow_matrix.loc['pi_d', 'B'] = - params['pi_dB']
    flow_matrix.loc['DeltaA', 'B'] = params['DeltaA_B']
    flow_matrix.loc['DeltaB', 'B'] = - params['DeltaB_B']
    flow_matrix.loc['DeltaM', 'B'] = - params['DeltaM_B']
    flow_matrix.loc['DeltaL', 'B'] = - params['DeltaL_B']
    flow_matrix.loc['DeltaD', 'B'] = params['DeltaD_B']
    flow_matrix.loc['W', 'G'] = - params['W_G']
    flow_matrix.loc['Z', 'G'] = - params['Z_G']
    flow_matrix.loc['T', 'G'] = params['T_G']
    flow_matrix.loc['iota_B', 'G'] = - params['iota_BG']
    flow_matrix.loc['pi', 'G'] = params['pi_G']
    flow_matrix.loc['DeltaB', 'G'] = params['DeltaB_G']
    flow_matrix.loc['DeltaM', 'G'] = - params['DeltaM_G']
    flow_matrix.loc['iota_A', 'CB'] = params['iota_ACB']
    flow_matrix.loc['iota_B', 'CB'] = params['iota_BCB']
    flow_matrix.loc['pi', 'CB'] = - params['pi_CB']
    flow_matrix.loc['DeltaA', 'CB'] = - params['DeltaA_CB']
    flow_matrix.loc['DeltaB', 'CB'] = - params['DeltaB_CB']
    flow_matrix.loc['DeltaM', 'CB'] = params['DeltaM_CB']
    flow_matrix.loc['sigma', :] = flow_matrix.sum()
    flow_matrix['sigma'] = flow_matrix.sum(axis=1)
    return stock_matrix, flow_matrix

In [10]:
params = {'Z_i':2, 'Z_j':2}
params = fill_initial_params(params)
params.update({'M_H12':10, 'M_F12':20, 'L_F1':50, 'M_B':10})
print_matrices(calc_initial_matrices(params))

stocks
           H     F     B    G   CB  sigma
M      10.0  20.0  10.0  0.0  0.0   40.0
A       0.0   0.0   0.0  0.0  0.0    0.0
D       0.0   0.0   0.0  0.0  0.0    0.0
B       0.0   0.0   0.0  0.0  0.0    0.0
L       0.0 -50.0   0.0  0.0  0.0  -50.0
V     -10.0  30.0 -10.0 -0.0 -0.0   10.0
sigma   0.0   0.0   0.0  0.0  0.0    0.0

flows
           H    F    B    G   CB  sigma
C       0.0  0.0  0.0  0.0  0.0    0.0
W       0.0  0.0  0.0  0.0  0.0    0.0
Z       0.0  0.0  0.0  0.0  0.0    0.0
T       0.0  0.0  0.0  0.0  0.0    0.0
iota_A  0.0  0.0  0.0  0.0  0.0    0.0
iota_B  0.0  0.0  0.0  0.0  0.0    0.0
iota_L  0.0  0.0  0.0  0.0  0.0    0.0
iota_D  0.0  0.0  0.0  0.0  0.0    0.0
pi_d    0.0  0.0  0.0  0.0  0.0    0.0
pi      0.0  0.0  0.0  0.0  0.0    0.0
DeltaA  0.0  0.0  0.0  0.0  0.0    0.0
DeltaB  0.0  0.0  0.0  0.0  0.0    0.0
DeltaM  0.0  0.0  0.0  0.0  0.0    0.0
DeltaL  0.0  0.0  0.0  0.0  0.0    0.0
DeltaD  0.0  0.0  0.0  0.0  0.0    0.0
sigma   0.0  0.0  0.0  0.0  0.0 

## fonctions de controle des parametres

In [11]:
def check_initial_params(params):
    assert params is not None 

## fonctions de calibration par blocs

In [59]:
params = {
    'g_ss':0.5,                     # taux de croissance
    'N_E1':5,                       # nombre de entrepreneurs dans le secteur productif 1
    'N_E2':6,                       # nombre de entrepreneurs dans le secteur productif 2
    'N_E3':7,                       # nombre de entrepreneurs dans le secteur productif 3
    'N_W1':10,                      # nombre de salaries dans le secteur productif 1
    'N_W2':20,                      # nombre de salaries dans le secteur productif 2
    'N_W3':30,                      # nombre de salaries dans le secteur productif 3
    'N_WG':40,                      # nombre de salaries dans le secteur public
    'N_U':40,                       # nombre de chomeurs dans le secteur public
    'phi1':1.1,                     # productivite initial du secteur productif 1
    'phi2':1.4,                     # productivite initial du secteur productif 2
    'phi3':1.2,                     # productivite initial du secteur productif 3
    'w1':2.5,                       # salaire initial du secteur productif 1
    'w2':2,                         # salaire initial du secteur productif 2
    'w3':1,                         # salaire initial du secteur productif 3
    'w_G':2,                        # salaire public
    'w_min':2,                      # salaire minimum
    'tau': 0.2,                     # taux d'impots
    'rho': 0.25,                    # politique de dividende
    'm':0.25,                       # taux de marge brute
    'alpha_b1':0.25,                # propension a consommer le bien 1 en zone urbaine
    'alpha_a2':0.25,                # propension a consommer le bien 2 en zone rurale
    'theta_W':0.75,                 # proportion desire de fonds de salaire
    'theta_E':0.75,                 # proportion desire de capitaux bancaire (rentabilite)
    'theta_Ubar':0.75,              # proportion reglementaire des allocations publics
    'r_D': 0.2,                     # taux d'interet sur les depots bancaires
    'r_L': 0.7,                     # taux d'interet sur les credits bancaires
    'r_B': 0.5,                     # taux d'interet sur les bons du tresors
    'r_A': 0.1,                     # taux d'interet sur les avances de la Banque Centrale
}

In [77]:
def calc_block_1(params):
    p = params.copy()
    p['zeta_1'] = 1 / (1 + p['g_ss'])
    p['zeta_2'] = 1 - p['zeta_1']
    
    for s in sectors:
        N = p[f'N_E{s}'] + p[f'N_W{s}']
        p[f'y{s}'] = p[f'phi{s}'] * N
        p[f'W_F{s}'] = p[f'w{s}'] * N
        p[f'p{s}'] = (1 + p['m']) * p[f'W_F{s}'] / p[f'y{s}']
        p[f'Q{s}'] = p[f'p{s}'] * p[f'y{s}']
        
        if s in [1, 3]:
            p[f'L_F{s}'] = 0
            p[f'D_F{s}'] = 0
            p[f'T_F{s}'] = 0
            p[f'pi_F{s}'] = p[f'Q{s}'] - p[f'W_F{s}']
            p[f'pi_dF{s}'] = p['rho'] - p[f'pi_F{s}']
            p[f'M_F{s}'] = (p[f'pi_F{s}'] - p[f'pi_dF{s}'])/ p['zeta_2']

        else:
            p['M_F2'] = 0
            p['D_F2'] = p['theta_W'] * p['W_F2']
            A = np.array([
                [1, 0, 0, p['zeta_1'] * p['r_L']],
                [p['tau'], -1, 0, 0],
                [p['rho'], -p['rho'], -1, 0],
                [1, -1, -1, p['zeta_2']]
            ])
            B = np.array([
                p['Q2'] + p['zeta_1'] * p['r_D'] * p['D_F2'] - p['W_F2'],
                0,
                0,
                p['zeta_2'] * p['D_F2']
            ])
            X = np.linalg.solve(A, B)
            p['pi_F2'] = X[0]
            p['T_F2'] = X[1]
            p['pi_dF2'] = X[2]
            p['L_F2'] = X[3]
        
        p[f'iota_LF{s}'] = p['zeta_1'] * p['r_L'] * p[f'L_F{s}']
        p[f'iota_DF{s}'] = p['zeta_1'] * p['r_D'] * p[f'D_F{s}']
        p[f'DeltaL_F{s}'] = p['zeta_2'] * p[f'L_F{s}']
        p[f'DeltaD_F{s}'] = p['zeta_2'] * p[f'D_F{s}']
        p[f'DeltaM_F{s}'] = p['zeta_2'] * p[f'M_F{s}'] 
    return p

In [78]:
p0 = fill_initial_params(params)
check_initial_params(p0)
print_matrices(calc_initial_matrices(p0))

stocks
          H    F    B    G   CB  sigma
M      0.0  0.0  0.0  0.0  0.0    0.0
A      0.0  0.0  0.0  0.0  0.0    0.0
D      0.0  0.0  0.0  0.0  0.0    0.0
B      0.0  0.0  0.0  0.0  0.0    0.0
L      0.0  0.0  0.0  0.0  0.0    0.0
V     -0.0 -0.0 -0.0 -0.0 -0.0    0.0
sigma  0.0  0.0  0.0  0.0  0.0    0.0

flows
           H    F    B    G   CB  sigma
C       0.0  0.0  0.0  0.0  0.0    0.0
W       0.0  0.0  0.0  0.0  0.0    0.0
Z       0.0  0.0  0.0  0.0  0.0    0.0
T       0.0  0.0  0.0  0.0  0.0    0.0
iota_A  0.0  0.0  0.0  0.0  0.0    0.0
iota_B  0.0  0.0  0.0  0.0  0.0    0.0
iota_L  0.0  0.0  0.0  0.0  0.0    0.0
iota_D  0.0  0.0  0.0  0.0  0.0    0.0
pi_d    0.0  0.0  0.0  0.0  0.0    0.0
pi      0.0  0.0  0.0  0.0  0.0    0.0
DeltaA  0.0  0.0  0.0  0.0  0.0    0.0
DeltaB  0.0  0.0  0.0  0.0  0.0    0.0
DeltaM  0.0  0.0  0.0  0.0  0.0    0.0
DeltaL  0.0  0.0  0.0  0.0  0.0    0.0
DeltaD  0.0  0.0  0.0  0.0  0.0    0.0
sigma   0.0  0.0  0.0  0.0  0.0    0.0


In [79]:
p1 = calc_block_1(p0)
print_matrices(calc_initial_matrices(p1))


stocks
          H       F    B    G   CB   sigma
M      0.0  110.25  0.0  0.0  0.0  110.25
A      0.0    0.00  0.0  0.0  0.0    0.00
D      0.0   39.00  0.0  0.0  0.0   39.00
B      0.0    0.00  0.0  0.0  0.0    0.00
L      0.0  -39.00  0.0  0.0  0.0  -39.00
V     -0.0 -110.25 -0.0 -0.0 -0.0 -110.25
sigma  0.0    0.00  0.0  0.0  0.0    0.00

flows
           H       F    B    G   CB   sigma
C       0.0  158.12  0.0  0.0  0.0  158.12
W       0.0 -126.50  0.0  0.0  0.0 -126.50
Z       0.0    0.00  0.0  0.0  0.0    0.00
T       0.0   -0.00  0.0  0.0  0.0   -0.00
iota_A  0.0    0.00  0.0  0.0  0.0    0.00
iota_B  0.0    0.00  0.0  0.0  0.0    0.00
iota_L  0.0  -18.20  0.0  0.0  0.0  -18.20
iota_D  0.0    5.20  0.0  0.0  0.0    5.20
pi_d    0.0   18.12  0.0  0.0  0.0   18.12
pi      0.0    0.00  0.0  0.0  0.0    0.00
DeltaA  0.0    0.00  0.0  0.0  0.0    0.00
DeltaB  0.0    0.00  0.0  0.0  0.0    0.00
DeltaM  0.0  -36.75  0.0  0.0  0.0  -36.75
DeltaL  0.0   13.00  0.0  0.0  0.0   13.00
Del

In [80]:
def calc_block_2(params):
    p = params.copy()
    for z in regions:
        if z == 'a':
            p['D_Ha'] = 0
            p['T_Ha'] = 0
            p['Z_Ha'] = 0
            p['W_Ha'] = p['W_F1']
            p['pi_dHa'] = p['pi_dF1']
            p['Ya'] = p['W_Ha'] + p['pi_dHa']
            p['Y_da'] = p['Ya']
            p['Ca'] = (1 - p['alpha_b1']) * p['Q1'] + p['alpha_a2'] * p['Q2']
            p['M_Ha'] = (p['Y_da'] - p['Ca']) / p['zeta_2']

        else:
            p['M_Hb'] = 0
            p['Cb'] = p['alpha_b1'] * p['Q1'] + (1 - p['alpha_a2']) * p['Q2'] + p['Q3']
            p['W_G'] = p['w_G'] * p['N_WG']
            p['W_Hb'] = p['W_F2'] + p['W_F3'] + p['W_G']
            p['Z_Hb'] = p['theta_Ubar'] * p['w_min'] * p['N_U']
            p['pi_dB'] = p['rho'] * p['zeta_1'] * p['r_L'] * p['L_F2']
            p['pi_dHb'] = p['pi_dF2'] + p['pi_dF3'] + p['pi_dB']
            A = np.array([
                [1, 0, 0, -p['zeta_1'] * p['r_D']],
                [p['tau'], -1, 0, 0],
                [1, -1, -1, 0],
                [0, 0, 1, -p['zeta_2']]
            ])
            B = np.array([
                p['W_Hb'] + p['pi_dHb'] + p['Z_Hb'],
                p['tau'] * p['Z_Hb'],
                0,
                p['Cb']
            ])
            X = np.linalg.solve(A, B)
            p['Yb'] = X[0]
            p['T_Hb'] = X[1]
            p['Y_db'] = X[2]
            p['D_Hb'] = X[3]
            
        p[f'iota_DH{z}'] = p['zeta_1'] * p['r_D'] * p[f'D_H{z}']
        p[f'DeltaD_H{z}'] = p['zeta_2'] * p[f'D_H{z}']
        p[f'DeltaM_H{z}'] = p['zeta_2'] * p[f'M_H{z}']     
    return p
        

In [81]:
p2 = calc_block_2(p1)
print_matrices(calc_initial_matrices(p2))

stocks
             H       F    B    G   CB   sigma
M      -69.09  110.25  0.0  0.0  0.0   41.16
A        0.00    0.00  0.0  0.0  0.0    0.00
D      374.65   39.00  0.0  0.0  0.0  413.65
B        0.00    0.00  0.0  0.0  0.0    0.00
L        0.00  -39.00  0.0  0.0  0.0  -39.00
V     -305.56 -110.25 -0.0 -0.0 -0.0 -415.81
sigma    0.00    0.00  0.0  0.0  0.0    0.00

flows
              H       F     B     G   CB   sigma
C      -158.12  158.12  0.00   0.0  0.0    0.00
W       206.50 -126.50  0.00 -80.0  0.0    0.00
Z        60.00    0.00  0.00   0.0  0.0   60.00
T       -42.90   -0.00  0.00   0.0  0.0  -42.90
iota_A    0.00    0.00  0.00   0.0  0.0    0.00
iota_B    0.00    0.00  0.00   0.0  0.0    0.00
iota_L    0.00  -18.20  0.00   0.0  0.0  -18.20
iota_D   49.95    5.20  0.00   0.0  0.0   55.15
pi_d    -13.58   18.12 -4.55   0.0  0.0   -0.00
pi        0.00    0.00  0.00   0.0  0.0    0.00
DeltaA    0.00    0.00  0.00   0.0  0.0    0.00
DeltaB    0.00    0.00  0.00   0.0  0.0    0.00


In [82]:
def calc_block_3(params):
    p = params.copy()
    p['A_B'] = 0
    p['D_B'] = p['D_Hb'] + p['D_F2']
    p['L_B'] = p['L_F2']
    A = np.array([
        [1, 0, 0, -p['zeta_1'] * p['r_B'], 0],
        [p['tau'], -1, 0, 0, 0],
        [p['rho'], -p['rho'], 0, 0, 0],
        [0, 0, 1, -1, -1],
        [1, -1, 0, -p['zeta_2'], -p['zeta_2']]
    ])
    B = np.array([
        p['zeta_1'] * p['r_L'] * p['L_B'] - p['zeta_1'] * p['r_D'] * p['D_B'],
        0,
        p['pi_dB'],
        p['L_B'] - p['D_B'],
        p['pi_dB'] + p['zeta_2'] * p['L_B'] - p['zeta_2'] * p['D_B']
    ])
    X = np.linalg.solve(A, B)
    p['pi_B'] = X[0]
    p['T_B'] = X[1]
    p['E_B'] = X[2]
    p['B_B'] = X[3]
    p['M_B'] = X[4]
    print(p['zeta_2'] * p['M_B'])
    
    p['iota_LB'] = p['zeta_1'] * p['r_L'] * p['L_B']
    p['iota_DB'] = p['zeta_1'] * p['r_D'] * p['D_B']
    p['iota_BB'] = p['zeta_1'] * p['r_B'] * p['B_B']
    p['DeltaB_B'] = p['zeta_2'] * p['B_B']
    p['DeltaL_B'] = p['zeta_2'] * p['L_B'] 
    p['DeltaD_B'] = p['zeta_2'] * p['D_B'] 
    p['DeltaM_B'] = p['zeta_2'] * p['M_B']
    return p
    

In [83]:
p3 = calc_block_3(p2)
print_matrices(calc_initial_matrices(p3))

78.83051470588235
stocks
             H       F       B    G   CB   sigma
M      -69.09  110.25  236.49  0.0  0.0  277.65
A        0.00    0.00    0.00  0.0  0.0    0.00
D      374.65   39.00 -413.65  0.0  0.0    0.00
B        0.00    0.00  179.11  0.0  0.0  179.11
L        0.00  -39.00   39.00  0.0  0.0    0.00
V     -305.56 -110.25  -40.95 -0.0 -0.0 -456.76
sigma    0.00    0.00    0.00  0.0  0.0    0.00

flows
              H       F       B     G   CB  sigma
C      -158.12  158.12    0.00   0.0  0.0   0.00
W       206.50 -126.50    0.00 -80.0  0.0   0.00
Z        60.00    0.00    0.00   0.0  0.0  60.00
T       -42.90   -0.00   -4.55   0.0  0.0 -47.45
iota_A    0.00    0.00    0.00   0.0  0.0   0.00
iota_B    0.00    0.00   59.70   0.0  0.0  59.70
iota_L    0.00  -18.20   18.20   0.0  0.0   0.00
iota_D   49.95    5.20  -55.15   0.0  0.0   0.00
pi_d    -13.58   18.12   -4.55   0.0  0.0  -0.00
pi        0.00    0.00    0.00   0.0  0.0   0.00
DeltaA    0.00    0.00    0.00   0.0  0.0  

In [84]:
def calc_block_4(params):
    p = params.copy()
    p['Z_G'] = p['Z_Ha'] + p['Z_Hb']
    p['A_CB'] = p['M_G'] = 0
    p['T_G'] = p['T_Hb'] + p['T_F2'] + p['T_B']
    p['M_F'] = p['M_F1'] + p['M_F2'] + p['M_F3']
    p['M_H'] = p['M_Ha'] + p['M_Ha']
    A = np.array([
        [1, -1, 0, 0, 0],
        [0, 0, 1, 0, -1],
        [0, 0, 0, 1, -1],
        [1, 0, 0, p['zeta_2'] - p['zeta_1'] * p['r_B'], 0],
        [0, 1, 0, 0, -p['zeta_1'] * p['r_B']]
    ])
    B = np.array([
        0,
        0,
        p['B_B'],
        p['W_G'] + p['Z_G'] - p['T_G'],
        0
    ])
    X = np.linalg.solve(A, B)
    p['pi_G'] = X[0]
    p['pi_CB'] = X[1]
    p['M_CB'] = X[2]
    p['B_G'] = X[3]
    p['B_CB'] = X[4]
    
    p['iota_BG'] = p['zeta_1'] * p['r_B'] * p['B_G']
    p['iota_BCB'] = p['zeta_1'] * p['r_B'] * p['B_CB']
    p['DeltaB_G'] = p['zeta_2'] * p['B_G']
    p['DeltaB_CB'] = p['zeta_2'] * p['B_CB']
    p['DeltaM_G'] = p['zeta_2'] * p['M_G']
    p['DeltaM_CB'] = p['zeta_2'] * p['M_CB']
    return p

In [85]:
p4 = calc_block_4(p3)
print_matrices(calc_initial_matrices(p4))

stocks
             H      F       B       G      CB  sigma
M     -207.28  220.5  236.49    0.00 -277.65 -27.94
A        0.00    0.0    0.00    0.00    0.00   0.00
D      374.65   39.0 -413.65    0.00    0.00   0.00
B        0.00    0.0  179.11 -456.76  277.65   0.00
L        0.00  -39.0   39.00    0.00    0.00   0.00
V     -167.37 -220.5  -40.95  456.76   -0.00  27.94
sigma    0.00    0.0    0.00    0.00    0.00   0.00

flows
              H       F       B       G     CB  sigma
C      -158.12  158.12    0.00    0.00   0.00    0.0
W       206.50 -126.50    0.00  -80.00   0.00    0.0
Z        60.00    0.00    0.00  -60.00   0.00    0.0
T       -42.90   -0.00   -4.55   47.45   0.00    0.0
iota_A    0.00    0.00    0.00    0.00   0.00    0.0
iota_B    0.00    0.00   59.70 -152.25  92.55    0.0
iota_L    0.00  -18.20   18.20    0.00   0.00    0.0
iota_D   49.95    5.20  -55.15    0.00   0.00    0.0
pi_d    -13.58   18.12   -4.55    0.00   0.00   -0.0
pi        0.00    0.00    0.00   92.55

## fonctions de distribution des stocks et flux

In [86]:
model = ap.Model()
households = ap.AgentList(model, 10)
households

AgentList (10 objects)

In [87]:
owners = households.random(2)
workers = households.random(7)
households.s_U = 1
households.s_W = 0
households.s_E = 0
owners.s_E = 1
owners.s_U = 0
workers.s_W = 1
workers.s_U = 0
sum(households.s_U), sum(households.s_W), sum(households.s_E)

(2, 7, 2)

In [88]:
households.x = 0
subworkers = workers.to_list().random(5)
subworkers.x = 5
sum(households.x)

25

In [89]:
agg_var(p4, 'N_E')

18

In [132]:
def create_households(m):
    p = m.p
    bankowners = ap.AgentList(m, 1)
    bankowners.s_EB = 1
    bankowners.s_E = 1
    m.households = bankowners
    
    firmowners1 = ap.AgentList(m, p['N_E1'])
    firmowners1.s_EB = 0
    firmowners1.s_E = 1
    m.households += firmowners1
    
    firmowners2 = ap.AgentList(m, p['N_E2'])
    firmowners2.s_EB = 0
    firmowners2.s_E = 1
    m.households += firmowners2
    
    firmowners3 = ap.AgentList(m, p['N_E3'])
    firmowners3.s_EB = 0
    firmowners3.s_E = 1
    m.households += firmowners3
    
    # firmowners = firmowners1 + firmowners2 + firmowners3
    # owners = bankowners + firmowners
    # print(len(firmowners), len(owners))
    # print(len(owners.select(owners.s_EB==0).select(owners.s_E==1)))
    # print(list(owners.select(owners.s_EB==0).select(owners.s_E==1)))
    
    privateworkers1 = ap.AgentList(m, p['N_W1'])
    privateworkers1.s_EB = 0
    privateworkers1.s_E = 0
    
    privateworkers2 = ap.AgentList(m, p['N_W2'])
    privateworkers2.s_EB = 0
    privateworkers2.s_E = 0
    
    privateworkers3 = ap.AgentList(m, p['N_W3'])
    privateworkers3.s_EB = 0
    privateworkers3.s_E = 0
    
    # privateworkers = privateworkers1 + privateworkers2 + privateworkers3 
    
    publicworkers = ap.AgentList(m, p['N_WG'])
    publicworkers.s_EB = 0
    publicworkers.s_E = 0
    # workers = privateworkers + publicworkers
    # print(list(privateworkers1))
    
    unemployed = ap.AgentList(m, p['N_U'])
    unemployed.s_EB = 0
    unemployed.s_E = 0
    
    # m.households = owners + workers + unemployed
    # print(len(households.select(households.s_EB==0).select(households.s_E==1)))
    # print(list(households.select(households.s_EB==0).select(households.s_E==1)))
    return m

In [133]:
model = ap.Model(p4)
model = create_households(model)
households = model.households
print('Num households', agg_var(p4, 'N') + 1, len(households))
print('Num bank owners', 1, len(households.select(households.s_EB == 1)))
print('Num firm owners', agg_var(p4, 'N_E'), len(households.select(households.s_EB==0).select(households.s_E==1)))

Num households 159 19
Num bank owners 1 1
Num firm owners 18 18


In [110]:
a1 = ap.AgentList(model, 2)
a2 = ap.AgentList(model, 5)
a1 + a2

AgentList (7 objects)